<a href="https://colab.research.google.com/github/NavdeepTU/Deep-Learning/blob/main/02.%20ML%20basics%20with%20Keras/07.%20Save%20and%20load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyyaml h5py # Required to save models in HDF5 format

In [2]:
import os
import tensorflow as tf

In [6]:
# to demonstrate how to save and load weights, we'll use the MNIST dataset.
# To speed up these runs, use the first 1000 examples.

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [8]:
# Define a model

# define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(512, activation = 'relu', input_shape = (784,)),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10)
  ])

  model.compile(optimizer = 'adam',
                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
# Save checkpoints during training

# You can use a trained model without having to retrain it,
# or pick-up training where you left off in case the training process was interrupted.

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                 save_weights_only = True,
                                                 verbose = 1)

# Train the model with new callback
model.fit(train_images,
          train_labels,
          epochs = 10,
          validation_data = (test_images, test_labels),
          callbacks = [cp_callback]) # Pass callback to training

Epoch 1/10
28/32 [=========================>....] - ETA: 0s - loss: 1.2599 - sparse_categorical_accuracy: 0.6317
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 2s 28ms/step - loss: 1.1917 - sparse_categorical_accuracy: 0.6530 - val_loss: 0.7035 - val_sparse_categorical_accuracy: 0.7980
Epoch 2/10
27/32 [========================>.....] - ETA: 0s - loss: 0.4073 - sparse_categorical_accuracy: 0.8866
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 17ms/step - loss: 0.4116 - sparse_categorical_accuracy: 0.8820 - val_loss: 0.5332 - val_sparse_categorical_accuracy: 0.8330
Epoch 3/10
28/32 [=========================>....] - ETA: 0s - loss: 0.2938 - sparse_categorical_accuracy: 0.9230
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 17ms/step - loss: 0.2872 - sparse_categorical_accuracy: 0.9250 - val_loss: 0.4942 - val_sparse_categorical_accuracy: 0.8410
Epoch 4/10
31/32 [=======

In [10]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'cp.ckpt.index', 'checkpoint']

In [11]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print("Untrained model accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3458 - sparse_categorical_accuracy: 0.1460 - 255ms/epoch - 8ms/step
Untrained model accuracy: 14.60%


In [12]:
# Load the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4113 - sparse_categorical_accuracy: 0.8590 - 200ms/epoch - 6ms/step
Restored model, accuracy: 85.90%
